In [ ]:
import numpy as np
import matplotlib.pyplot as plt

### Adjacency matrix for grid

In [ ]:
def grid_adj_mat(I, J):
    '''Generates an adjacency matrix repesenting a grid.
    
    NOTE: Assumes 8-directional connections, and that the
    grid origo is in the top-left corner.
    
    Args:
        I (int): Row count
        J (int): Col count 
    '''
    # Number of elements
    n = I * J
    A = np.zeros((n, n), dtype=np.int8)
    
    diag_block = np.zeros((J, J), dtype=np.int8)
    for idx in range(J-1):
        diag_block[idx, idx+1] = 1
        diag_block[idx+1, idx] = 1
    
    side_block = np.eye(J, dtype=np.int8) + diag_block
    
    # First block row
    if I == 1:
        A[0:J, 0:J] = diag_block
    else:
        A[0:J, 0:J] = diag_block
        A[0:J, J:2*J] = side_block
        
        # Last block row
        A[-1*J:, -2*J:-1*J] = side_block
        A[-1*J:, -1*J:] = diag_block
    
    # Middle block rows
    for idx in range(1,I-1):
        i_start = idx*J
        i_end = (idx+1)*J
        A[i_start:i_end, (idx-1)*J:(idx+0)*J] = side_block
        A[i_start:i_end, (idx+0)*J:(idx+1)*J] = diag_block
        A[i_start:i_end, (idx+1)*J:(idx+2)*J] = side_block
    
    return A

# Grid dimension
I = 10
J = 15
A = grid_adj_mat(I, J)
plt.imshow(A)
plt.show()

In [ ]:
def get_neighbor_nodes(node_idx, A):
    '''Returns a list of node indices corresponing to neighbors of given node.
    '''
    return np.nonzero(A[node_idx, :])[0]

def breadth_first_search(A, node_s_idx):
    '''Returns arrays of distance and parent vertices from a starting vertex.
    Args:
        A: Adjacency matrix (n, n)
        node_s_idx (int): Index of starting node.
    Returns:
        dist: Node idx ordered list with edge distances from starting vertex.
        par: Node idx ordered list with parent vertices.
    '''
    # Node count
    N = A.shape[0]

    # Initialization
    color = np.zeros(N, dtype=np.uint8)
    dist = np.zeros(N, dtype=np.uint32)
    par = -1*np.ones(N, dtype=np.uint32)  # Unconnected nodes: -1

    # 'First-in, first-out' queue for storing neighbor vertices
    Q = []

    # Add starting node
    Q.append(node_s_idx)

    while len(Q) > 0:

        u = Q.pop(0)

        # Extract list of neighbor vertex indices from adjacency matrix row
        neigh_vert_set = get_neighbor_nodes(u, A)

        for v in neigh_vert_set:

            if color[v] == 0:  # 'White' undiscovered node

                color[v] = 1  # 'Gray' discovered node
                dist[v] = dist[u] + 1
                par[v] = u

                Q.append(v)

        color[u] = 2  # 'Black' finished node

    return dist, par

In [ ]:
dist, par = breadth_first_search(A, 0)

print(par)

dist_arr = dist.reshape((I,J))
plt.imshow(dist_arr)
plt.show()

In [ ]:
goal_idx = I*J-1
path = []

par_idx = goal_idx
while True:
    
    path.insert(0,  par_idx)
    par_idx = par[par_idx]
    if par_idx == -1:
        break

print(path)

# Convert vertice idx to (i, j) coordinates
i = []
j = []
for path_vert in path:
    i.append( int(path_vert / J) )
    j.append( path_vert % J)

route = np.zeros((I,J))
for idx in range(len(i)):
    route[i[idx], j[idx]] = 1.0

plt.imshow(route)
plt.show()